# Functions

In [2]:
square x = x*x

In [3]:
plus x y = x+y

In [4]:
inc = plus 1

In [6]:
compose f g = \x -> f(g(x))

In [7]:
($) = compose

In [8]:
ps = square $ inc

In [9]:
ps 4

25

In [10]:
flip f x y = f y x

In [11]:
(.) = flip compose

In [12]:
ps = inc . square

In [13]:
ps 4

25

In [14]:
(>>) x f = f x

In [15]:
4 >> square

16

In [16]:
4 >> inc >> square

25

# Maybe Type

In [17]:
data Maybe a = Just a | Nothing deriving IHaskellPrelude.Show

In [18]:
div x 0 = Nothing
div x y = Just (x IHaskellPrelude./ y)

In [19]:
div 10 4

Just 2.5

In [20]:
div 10 0

Nothing

# List Type

In [21]:
data List a = L a (List a) | Empty deriving IHaskellPrelude.Show

In [22]:
l = L 1 (L 2 (L 3 Empty))

In [23]:
l

L 1 (L 2 (L 3 Empty))

In [24]:
reduce f p Empty = p
reduce f p (L h t) = f h (reduce f p t)

In [25]:
sum = reduce plus 0

In [26]:
sum l

6

In [27]:
head Empty = Nothing
head (L h t) = Just h

In [28]:
head l

Just 1

In [29]:
tail Empty = Nothing
tail (L h t) = Just t

In [30]:
tail l

Just (L 2 (L 3 Empty))

In [31]:
tail (L 23 Empty)

Just Empty

In [32]:
second x = head (tail x)

Couldn't match expected type `List a' with actual type `Maybe (List t)'
Relevant bindings include
  x :: List t (bound at <interactive>:1:8)
  second :: List t -> Maybe a (bound at <interactive>:1:1)
In the first argument of `head', namely `(tail x)'
In the expression: head (tail x)

In [33]:
:t tail

tail :: forall t. List t -> Maybe (List t)

In [34]:
:t head

head :: forall a. List a -> Maybe a

In [35]:
mhead Nothing = Nothing
mhead (Just x) = head x

In [36]:
mtail Nothing = Nothing
mtail (Just x) = tail x

In [37]:
mtail (Just (L 1 Empty))

Just Empty

In [38]:
second x = mhead (mtail (Just x))

In [39]:
second l

Just 2

In [40]:
second Empty

Nothing

## Well, this is not practical to define all these "m" functions, no?

In [41]:
mbind f Nothing = Nothing
mbind f (Just x) = f x

In [42]:
mwrap x = Just x

In [43]:
second x = (mbind head) ((mbind tail ) (mwrap x))

In [44]:
second l

Just 2

In [45]:
second = mwrap . (mbind tail) . (mbind head)

In [46]:
second l

Just 2

In [47]:
mcompose f g = mwrap . (mbind g) . (mbind f)

In [48]:
second = mcompose head tail

In [49]:
second l

Just 2

In [50]:
(%) f g = mcompose g f

In [51]:
l >> (tail % tail % head)

Just 3

In [52]:
5 >> (mwrap % mwrap % mwrap)

Just 5

In [53]:
inc (second l)

Non type-variable argument in the constraint: Num (Maybe a)
(Use FlexibleContexts to permit this)
When checking that `it' has the inferred type
  it :: forall a. (Num a, Num (Maybe a)) => Maybe a

In [54]:
mmap f Nothing = Nothing
mmap f (Just x) = Just (f x)

In [55]:
mmap inc (second l)

Just 3

# More List Functions 

In [56]:
lmap f Empty = Empty
lmap f (L h t) = L (f h) (lmap f t)

In [57]:
lmap square l

L 1 (L 4 (L 9 Empty))

In [58]:
concat Empty Empty = Empty
concat x Empty = x
concat Empty x = x
concat (L h t) x = L h (concat t x)

In [59]:
concat l l

L 1 (L 2 (L 3 (L 1 (L 2 (L 3 Empty)))))

In [60]:
cdown 0 = L 0 Empty
cdown x = L x (cdown (x-1))

In [61]:
cdown 3

L 3 (L 2 (L 1 (L 0 Empty)))

In [62]:
lmap cdown l

L (L 1 (L 0 Empty)) (L (L 2 (L 1 (L 0 Empty))) (L (L 3 (L 2 (L 1 (L 0 Empty)))) Empty))

In [63]:
flatten Empty = Empty
flatten (L h t) = concat h (flatten t)

In [85]:
flatten (lmap cdown l)

L 1 (L 0 (L 2 (L 1 (L 0 (L 3 (L 2 (L 1 (L 0 Empty))))))))

In [86]:
flatmap f x = flatten (lmap f x)

In [87]:
flatmap cdown l

L 1 (L 0 (L 2 (L 1 (L 0 (L 3 (L 2 (L 1 (L 0 Empty))))))))

In [66]:
lwrap x = (L x Empty)

In [67]:
ccdown = lwrap . (flatmap cdown) . (flatmap cdown)

In [68]:
ccdown 3

L 3 (L 2 (L 1 (L 0 (L 2 (L 1 (L 0 (L 1 (L 0 (L 0 Empty)))))))))

In [69]:
lbind = flatmap

In [70]:
join f g = lwrap . (lbind f)  . (lbind g)

In [71]:
ccdown = join cdown cdown

In [72]:
ccdown 3

L 3 (L 2 (L 1 (L 0 (L 2 (L 1 (L 0 (L 1 (L 0 (L 0 Empty)))))))))

In [73]:
(&) f g = join g f

In [74]:
3 >> (cdown & cdown & cdown)

L 3 (L 2 (L 1 (L 0 (L 2 (L 1 (L 0 (L 1 (L 0 (L 0 (L 2 (L 1 (L 0 (L 1 (L 0 (L 0 (L 1 (L 0 (L 0 (L 0 Empty)))))))))))))))))))

In [75]:
3 >> (lwrap & lwrap & lwrap) 

L 3 Empty

# Even More List Functions

In [76]:
append Empty x = L x Empty
append (L h t) x = L h (append t x)

In [77]:
append l 4

L 1 (L 2 (L 3 (L 4 Empty)))

In [78]:
reverse Empty = Empty
reverse (L h t) = append (reverse t) h

In [79]:
reverse l

L 3 (L 2 (L 1 Empty))

In [80]:
cup = cdown . reverse

In [81]:
cup 6

L 0 (L 1 (L 2 (L 3 (L 4 (L 5 (L 6 Empty))))))

In [82]:
flatmap f Empty = Empty
flatmap f (L h t) = concat (f h) (flatmap f t)

In [83]:
flatmap cdown l

L 1 (L 0 (L 2 (L 1 (L 0 (L 3 (L 2 (L 1 (L 0 Empty))))))))